In [1]:
# Class Session 2

# this is the URL of the SIF file at Pathway Commons
sif_file_url <- "http://www.pathwaycommons.org/archives/PC2/current/PathwayCommons.8.All.BINARY_SIF.hgnc.txt.sif.gz"

# for starters, we only want three possible interaction types
interaction_types_ppi <- c("interacts-with","in-complex-with","neighbor-of")

In [7]:
# do the filtering while reading the data; reduce disk space and memory usage
library(readr)
system.time(interactions_df <- read_delim_chunked(sif_file_url,
                                     callback=DataFrameCallback$new(function(df_chunk, pos){ 
                                        subset(df_chunk, interaction_type %in% interaction_types_ppi)}),
                                     chunk_size=10000,
                                     delim="\t",
                                     quote="",
                                     comment="",
                                     col_names=c("species1","interaction_type","species2"),
                                     progress=FALSE))

# sanity check the resulting data frame
head(interactions_df)

Parsed with column specification:
cols(
  species1 = col_character(),
  interaction_type = col_character(),
  species2 = col_character()
)


   user  system elapsed 
  2.168   1.128   3.295 

species1,interaction_type,species2
A1BG,interacts-with,ABCC6
A1BG,interacts-with,ANXA7
A1BG,interacts-with,CDKN1A
A1BG,interacts-with,CRISP3
A1BG,interacts-with,GDPD1
A1BG,in-complex-with,GRB2


In [5]:
# how many rows are there in the protein-protein interaction edge-list?
nrow(interactions_df)

[1] 523498

In [6]:
# how many unique proteins are there in the interaction network?
length(unique(c(interactions_df$species1, interactions_df$species2)))

[1] 17020

In [7]:
# how many unique interacting protein pairs are there in the interaction network?
length(unique(apply(interactions_df[,c(1,3)], 1, function(my_pair) {
    paste(c(min(my_pair), "-", max(my_pair)),collapse="")
})))

[1] 491784

In [6]:
# alternative approach which is kind of slow and (transiently) memory-hungry; read the entire file into memory and 
# then process the text contents of the file, line by line

system.time({
alltext <- readLines(gzcon(url(sif_file_url)))
nlines <- length(alltext)
interactions_df4 <- data.frame(do.call(rbind, lapply(1:length(alltext),
                                         function(i) {
                                             split_line <- strsplit(alltext[i], "\t")[[1]]
                                             names(split_line) <- c("species1","interaction_type","species2")
                                             split_line
                                         })))
interactions_df4 <- subset(interactions_df4, interaction_type %in% interaction_types_ppi)
})
            
# sanity check that this slow approach at least gave us correct results
all(interactions_df4 == interactions_df)
rm(interactions_df4)
rm(alltext)

   user  system elapsed 
 26.148   0.212  26.795 

[1] TRUE

In [7]:
# quick and dirty approach which doesn't require looking up any newfangled commands, but transiently eats up
# disk space and memory
fname <- tempfile()
gzfname <- paste(c(fname, ".gz"),collapse="")
system.time({
   download.file(sif_file_url, destfile=gzfname)
   interactions_df3 <- subset(read.table(file=gzfile(gzfname),
                              sep="\t",
                              comment.char="",
                              quote="",
                              col.names=c("species1","interaction_type", "species2"),
                              stringsAsFactors=FALSE),
                           interaction_type %in% interaction_types_ppi)})
unlink(gzfname)

# sanity check the results
all(interactions_df3 == interactions_df)

# clean up
rm(interactions_df3)

   user  system elapsed 
  2.988   0.000   4.873 

[1] TRUE

In [12]:
# really quick-and-dirty approach using cURL and gunzip
system.time({
    system(paste(c("curl -s --compressed ", sif_file_url, " | gunzip > pc.sif"), collapse=""), 
       intern=TRUE)
my_df <- read.table(file="pc.sif",
                    sep="\t",
                    comment.char="",
                    quote="",
                    col.names=c("species1",
                                "interaction_type",
                                "species2"),
                    stringsAsFactors=FALSE)
interactions_df5 <- subset(my_df,
                          interaction_type %in% interaction_types_ppi)})

all(interactions_df5 == interactions_df)


print(sort(table(as.factor(my_df$interaction_type)), decreasing=TRUE))

   user  system elapsed 
  2.820   0.152   4.315 

[1] TRUE


                interacts-with                in-complex-with 
                        369895                         153603 
              chemical-affects             catalysis-precedes 
                        135268                         120948 
        controls-expression-of       controls-state-change-of 
                        110013                         106156 
        controls-production-of      consumption-controlled-by 
                         18482                          16816 
   controls-phosphorylation-of                used-to-produce 
                         15636                          13705 
         controls-transport-of                    reacts-with 
                          6960                           3607 
controls-transport-of-chemical 
                          2847 
